- [OAI](https://arxiv.org/help/oa/index) for metadata bulk-download. Updated nightly. *The OAI-PMH interface does not support selective harvesting based on submission date. The datestamps are designed to support incremental harvesting of updates on an ongoig basis.*
    * number of articles capped at 1000 per request
    * 503 after frequent requests
    * datestamp and date created are very far
- 09/23: scraped all set:math via metha. 133M in gunzip.  4hrs. 406 xml.gz files.
- arXiv API for real-time metadata
- PDF bulk-download available on [S3](https://arxiv.org/help/bulk_data_s3). 1 cent per GB.
- Warning: *"If you build indexes or tools based on the full-text you must link back to arXiv for downloads."*
- On arXiv bulk download https://arxiv.org/help/bulk_data

In [7]:
import feedparser
import requests
import xml.etree.ElementTree as ET
from datetime import datetime
import pandas as pd

In [2]:
def arxiv_api():
    baseurl = 'http://export.arxiv.org/api/query?search_query='
    query_string = 'all:electron&start=0&max_results=1'
    response = requests.get(baseurl + query_string)
    response = feedparser.parse(response.text)
    summary = 
    return 

SyntaxError: invalid syntax (<ipython-input-2-e44972e8523f>, line 6)

In [50]:
# http://betatim.github.io/posts/analysing-the-arxiv/
def oai_query(start_date, end_date):
    baseurl = 'http://export.arxiv.org/oai2?verb=ListRecords&'
    dates = ''.join(['from=', start_date, '&until=', end_date, '&'])
    metadata_format = 'metadataPrefix=arXiv&'
    category = 'set=math'
    queryurl = ''.join([baseurl, dates, metadata_format, category])
    response = requests.get(queryurl)
    #feedparser fails
    if response.status_code == 503:
        raise requests.ConnectionError('503. %s' % response.text)
    else:
        return response

In [13]:
d1 = oai_query('2015-01-01', '2015-02-01')

In [51]:
d2 = oai_query('2015-02-02', '2015-02-05')

ConnectionError: 503. <html><body>
<h1>Retry after 600 seconds</h1>
The arXiv OAI service is unavailable due to maintenance</body></html>


In [49]:
d2.text

'<html><body>\n<h1>Retry after 600 seconds</h1>\nThe arXiv OAI service is unavailable due to maintenance</body></html>\n'

In [45]:
d2.status_code == 503

True

In [26]:
def query_to_df(xml):
    data = ET.fromstring(xml.text)
    OAIurl = '{http://www.openarchives.org/OAI/2.0/}'
    arXivOAI = '{http://arxiv.org/OAI/arXiv/}'
    df = pd.DataFrame(columns=("title", "abstract", "categories", "created", "id", "doi"))
    # Element.iter(): depth-first iteration of tree
    for record in data.find(OAIurl+'ListRecords').findall(OAIurl+"record"):
        arxiv_id = record.find(OAIurl+'header').find(OAIurl+'identifier')
        datestamp = record.find(OAIurl+'header').find(OAIurl+'datestamp')
        datestamp = datetime.strptime(datestamp.text, '%Y-%m-%d')
        meta = record.find(OAIurl+'metadata')
        info = meta.find(arXivOAI+"arXiv")
        created = info.find(arXivOAI+"created").text
        created = datetime.strptime(created, "%Y-%m-%d")
        categories = info.find(arXivOAI+"categories").text

        # if there is more than one DOI use the first one
        # often the second one (if it exists at all) refers
        # to an eratum or similar
        doi = info.find(arXivOAI+"doi")
        if doi is not None:
            doi = doi.text.split()[0]
                
        contents = {'title': info.find(arXivOAI+"title").text,
                    'id': info.find(arXivOAI+"id").text,#arxiv_id.text[4:],
                    'abstract': info.find(arXivOAI+"abstract").text.strip(),
                    'created': created,
                    'datestamp': datestamp,
                    'categories': categories.split(),
                    'doi': doi,
                    }

        df = df.append(contents, ignore_index=True)
    return df
    

In [27]:
df1 = query_to_df(d1)

In [33]:
df2 = query_to_df(d2)

AttributeError: 'NoneType' object has no attribute 'findall'

In [28]:
df1['created'].head()

0   2007-04-12
1   2007-06-01
2   2007-06-05
3   2007-09-05
4   2007-10-20
Name: created, dtype: datetime64[ns]

In [29]:
df1['datestamp'].head()

0   2015-01-06
1   2015-01-20
2   2015-01-05
3   2015-01-27
4   2015-01-27
Name: datestamp, dtype: datetime64[ns]

In [65]:
for c in data:
    print(c.tag, ' ! ', c.attrib)

{http://www.openarchives.org/OAI/2.0/}responseDate  !  {}
{http://www.openarchives.org/OAI/2.0/}request  !  {'verb': 'ListRecords', 'until': '2015-02-01', 'from': '2015-01-01', 'metadataPrefix': 'arXiv', 'set': 'math'}
{http://www.openarchives.org/OAI/2.0/}ListRecords  !  {}


In [64]:
for c in data.find(OAIurl + 'ListRecords')[:20]:
    print(c.tag, ' ! ', c.attrib)

{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.openarchives.org/OAI/2.0/}record  !  {}
{http://www.

In [72]:
for c in data.find(OAIurl + 'ListRecords').find(OAIurl + 'record').iter():
    print(list(c))

[<Element '{http://www.openarchives.org/OAI/2.0/}header' at 0x7f9c8e5e7c28>, <Element '{http://www.openarchives.org/OAI/2.0/}metadata' at 0x7f9c8dfdb908>]
[<Element '{http://www.openarchives.org/OAI/2.0/}identifier' at 0x7f9c8dfdb728>, <Element '{http://www.openarchives.org/OAI/2.0/}datestamp' at 0x7f9c8dfdb7c8>, <Element '{http://www.openarchives.org/OAI/2.0/}setSpec' at 0x7f9c8dfdb868>]
[]
[]
[]
[<Element '{http://arxiv.org/OAI/arXiv/}arXiv' at 0x7f9c8dfdb958>]
[<Element '{http://arxiv.org/OAI/arXiv/}id' at 0x7f9c8dfdb9f8>, <Element '{http://arxiv.org/OAI/arXiv/}created' at 0x7f9c8dfdba48>, <Element '{http://arxiv.org/OAI/arXiv/}authors' at 0x7f9c8dfdba98>, <Element '{http://arxiv.org/OAI/arXiv/}title' at 0x7f9c8dfdbdb8>, <Element '{http://arxiv.org/OAI/arXiv/}categories' at 0x7f9c8dfdbe08>, <Element '{http://arxiv.org/OAI/arXiv/}msc-class' at 0x7f9c8dfdbe58>, <Element '{http://arxiv.org/OAI/arXiv/}journal-ref' at 0x7f9c8dfdbea8>, <Element '{http://arxiv.org/OAI/arXiv/}doi' at 0x7f9c

- Nothing interesting in 'feed'. Just record of query made.
- 

<Element '{http://www.openarchives.org/OAI/2.0/}OAI-PMH' at 0x7f9c9d030228>